#### チャットボット

In [1]:
import csv
# インポート
import numpy as np
import matplotlib.pyplot as plt
import random
 # プログレスバーをロード
# from tqdm import tqdm

# 日本語を使用できるようにする
# import japanize_matplotlib

# 形態素解析
# import MeCab
# 形態素解析用辞書
# import unidic

def answer2(input_line, article_arr):
    for i in range(len(article_arr)):
        for word in article_arr[i]:
            if word == '':
                break
            if word in input_line:
                if 2 <= len(word):
                    return i
    return None

def get_key_word(inputline, word_list):
    for i in range(len(word_list)):
        if len(word_list[i]) >= 2:
            if word_list[i] in inputline:
                return word_list[i]

def get_ngram(sentence, Ng=1):
    Ns = len(sentence)
    idxs = []
    for i in range(Ns):
        if i+Ng > Ns:
            break
        idxs.append( [i,(i+Ng) ] )
    res = []
    for idx in idxs:
        i = idx[0]
        j = idx[1]
        tmp=""
        for k in sentence[i:j]:
            tmp+=k
        res.append( tmp)
    return res

def unique(w):
    return list(set(w) )


def sample(i, trans, input_string_dict, input_string_list):
    key = np.random.choice(a=input_string_dict, size=1, p=trans[i])[0]
    return input_string_dict.index(key)

def moji_count(input_string_dict, input_string_list):
    mojis=[]
    counts=[]
    idx=0
    for moji in input_string_dict:
        n=0
        for tmp in input_string_list:
            if moji==tmp:
                n+=1
        idx+=1
        mojis.append(moji)
        counts.append(n)
    return mojis, counts

def Markov_chain(input_string_dict, input_string_list, Ng):
    # 連続出現頻度をカウントする (条件付き確率を統計的に推測)
    C1 = np.zeros((len(input_string_dict),len(input_string_dict)))
    for i in range(len(input_string_list)-Ng):
        words = f"{input_string_list[i]}{input_string_list[i+Ng]}"
        #print(words)
        for j in range(len(input_string_dict)):
            m1=input_string_dict[j]
            if words[:Ng] == m1:
                for k in range(len(input_string_dict)):
                    m2=input_string_dict[k]
                    #print(f"{m1}{m2}")
                    if words==f"{m1}{m2}":
                        C1[j][k]+=1
    return C1

def Markov_probability(C1, input_string_dict):
    trans1 = np.array(C1)
    for i in range(len(input_string_dict)):
        # EOS を除く処理
        if np.sum(trans1[i])>0: # EOS(終端文字)以外は必ず満たされる。
            trans1[i] = trans1[i]/np.sum(trans1[i]) # 統計確率に変換。
        else:
            pass
    return trans1

def say(input_string_dict, input_string_list, counts, trans1, key):
    # key = np.random.choice(a=input_string_dict, size=1, p=counts/np.sum(counts))[0]
    i = input_string_dict.index(key)
    text = ''
    Nmax = 100
    # print('さくら：', end='')
    for k in range(Nmax):
        # if i==(len(input_string_dict)-1):
        #     print("[EOS]")
        #     break
        if input_string_dict[i][0] == '。' or input_string_dict[i][0] == '.':
            text = text.replace('\n', '')
            text += 'らしいです。\n'
            text += '\n▼ 詳しくはこちら\n'
            return text
        elif input_string_dict[i][1] == '。' or input_string_dict[i][1] == '.':
            text = text.replace('\n', '')
            text = text + input_string_dict[i][0]+'らしいです。\n'
            text += '\n▼ 詳しくはこちら\n'
            return text
        else:
            text += input_string_dict[i]
        i=sample(i,trans1, input_string_dict, input_string_list)
    return text

def speak(input_string, start_word):
    Ng = 2 # 1,2,3 あたりでないと遅い
    EOS= "%%%%EOS%%%%" # 終端記号を定義
    start_word = start_word[:Ng]
    # チャットボット用の文章
    # file = open(path, "r")
    # input_string = file.read()

    input_string_list = get_ngram(input_string, Ng=Ng)
    input_string_dict = unique(input_string_list)
    # input_string_list.append(EOS) # 終端記号を追加
    # input_string_dict.append(EOS) # 終端記号を追加

    mojis, counts = moji_count(input_string_dict, input_string_list)

    C1 = Markov_chain(input_string_dict, input_string_list, Ng)

    trans1 = Markov_probability(C1, input_string_dict)

    text = say(input_string_dict, input_string_list, counts, trans1, start_word)

    return text

In [2]:
# CSVを読み込む
article_list = []
# CSVファイルを1行ずつ参照する,Data_エンタメ_国内_科学.csv,Book1.cs
with open('test.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)

    # 次の行に進んで処理する例
    for row in csv_reader:
        article_list.append(row)
article_list[0][0] = article_list[0][0].replace('\ufeff', '')
article_arr = np.array(article_list)

In [3]:
def ChatBot(inputline):
      # 回答へ
      line_num = answer2(inputline, article_arr[:, 2:])
      if line_num != None:

          # path = prule[line_num][0]
          start_word = get_key_word(inputline, article_arr[line_num][2:])
          if start_word != None:
              text = speak(article_arr[line_num][1], start_word)
              text += article_arr[line_num][0]
          else:
              text = '記事がありません。'
      else:
          text = '記事がありません。'
      # この下に関
      #print(inputline, '+', text)
      return text

In [4]:
# ChatBot初期実行

article_list = []
# CSVファイルを1行ずつ参照する,Data_エンタメ_国内_科学.csv,Book1.cs
with open('test.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)

    # 次の行に進んで処理する例
    for row in csv_reader:
        article_list.append(row)
article_list[0][0] = article_list[0][0].replace('\ufeff', '')
article_arr = np.array(article_list)

# エキスパートシステム実行部
botname = 'さくら'
print(f"{botname}: さて、どうしました？")

さくら: さて、どうしました？


### LINEBOT サーバー

In [ ]:
import getpass
# LINEのチャネルシークレット
LINE_CHANNEL_SECRET = getpass.getpass(prompt='LINEのチャネルシークレットキー：')
# LINEのチャネルアクセストークン
LINE_ACCESS_TOKEN = getpass.getpass(prompt='LINEのチャネルアクセストークン：')
# ngrokのAuthtoken
NGROK_AUTHTOKEN = getpass.getpass(prompt='ngrokのAuthtoken：')

In [6]:
%%capture
!pip install flask
!pip install line-bot-sdk
!pip install pyngrok

#ngrok認証
!ngrok authtoken $NGROK_AUTHTOKEN

In [7]:
#Webフックを立てる
import os

from pyngrok import ngrok
from pyngrok.conf import PyngrokConfig

os.system('kill -9 $(pgrep ngrok)')
webhook_url = ngrok.connect(addr='127.0.0.1:5000', pyngrok_config=PyngrokConfig(start_new_session=True))
print (webhook_url)

print('https://developers.line.biz/console/channel/1660792247/messaging-apiでWebhookを設定してください')

NgrokTunnel: "https://8ff1-34-125-45-14.ngrok-free.app" -> "http://127.0.0.1:5000"
https://developers.line.biz/console/channel/1660792247/messaging-apiでWebhookを設定してください


#### データ送信プログラム

In [8]:
from linebot.models.messages import Message
from flask import Flask, request, abort #Webアプリケーションフレームワーク

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage,
)

app = Flask(__name__)

#API
line_bot_api = LineBotApi(LINE_ACCESS_TOKEN)
#ハンドラー
handler = WebhookHandler(LINE_CHANNEL_SECRET) #LINEチャンネルシークレットをWebhookが参照するように

@app.route("/test") #Flask関連,[URL]/testを踏むと以下の関数：test()が実行される。
def test():
    return "TEST OK"

#HTTPリクエスト？
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        # 署名の検証を行い、成功した場合にhandleされたメソッドを呼び出す
        handler.handle(body, signature)
    except InvalidSignatureError:
        #アクセストークンを確認する用に通知
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)#リクエストのイベントを指定
def handle_message(event):

    inputline = event.message.text # Lineユーザーの入力

    # 回答
    text = ChatBot(inputline)

    # 出力
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=f"{text}"),#event.message.text（送信されたメッセージ）を送信
    )
    text = '' # リセット

if __name__ == '__main__':
    app.run() #実行

 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-8-397559178546>:17: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(LINE_ACCESS_TOKEN)
<ipython-input-8-397559178546>:19: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(LINE_CHANNEL_SECRET) #LINEチャンネルシークレットをWebhookが参照するように
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Jul/2023 19:24:00] "POST / HTTP/1.1" 200 -
<ipython-input-8-397559178546>:56: LineBotSd